# CSAF F16 Example

This notebook illustrates how to run CSAF on a model of the F16. Each system in CSAF is comprised of a set of components that communicate with each other by sending messages defined in the ROS format (http://wiki.ros.org/Messages).

### Creating messages

Before we can create components, we need to define the message formats that a component uses to communicate over. In this example, we'll look at the messages used by the F16 low-level controller (LLC). It receives state from the F16 plant and outputs a set of control signals. The cell below loads the state output message from the F16 plant. The `version_major`, `version_minor`, `topic` and `time` fields are required for all CSAF messages. The remainder of the values are state variables in the aircraft model. For example, `vt` is the air speed, `theta` is the pitch and `psi` is the yaw.

In [2]:
!cat /csaf-system/components/msg/f16plant_state.msg

uint32 version_major
uint32 version_minor

string topic

float64 time

float64 vt
float64 alpha
float64 beta
float64 phi
float64 theta
float64 psi
float64 p
float64 q
float64 r
float64 pn
float64 pe
float64 h
float64 pow

Now let's look at the message used to capture control signals from the low-level controller. In addition to the standard fields, the message defines `delta_e` for the elevator, `delta_a` for the ailerons, `delta_r` for the rudder and `throttle`. 


In [3]:
!cat /csaf-system/components/msg/f16llc_output.msg

uint32 version_major
uint32 version_minor

string topic

float64 time

float64 delta_e
float64 delta_a
float64 delta_r
float64 throttle


### Creating components

A component in CSAF is defined by a TOML configuration file and a model file. The configuration file defines the messages a component consumes, the messages it produces and the parameters of the model. Below is the configuration file for the F16 low-level controller.

In [4]:
!cat /csaf-system/components/f16llc.toml

system_name = "F16 Low Level Controller"
system_representation = "black box"
system_solver = "Euler"

sampling_frequency = 100

is_discrete = false
is_hybrid = false

[parameters]
  lqr_name = "lqr_original"
  throttle_max = 1
  throttle_min = 0
  elevator_max = 25
  elevator_min = -25
  aileron_max = 21.5
  aileron_min = -21.5
  rudder_max = 30.0
  rudder_min = -30.0

[inputs]
  msgs = [ "f16plant_state.msg", "f16plant_output.msg", "autopilot_output.msg" ]

[topics]

  [topics.outputs]
    msg = "f16llc_output.msg"

  [topics.states]
    msg = "f16llc_state.msg"
    initial = [ 0.0, 0.0, 0.0 ]


Next we need to define the actual implementation of the low-level controller component. CSAF provides a very concise mechanism for doing so. All of the logic needed to generate, serialize and transport ROS messages is handled by the framework itself. This allows component implementations to focus on the core control logic by defining one of more of the following methods `model_output`, `model_state_update`, `model_info` and `model_update`. The full implementation for LLC is below. 

In [5]:
!cat /csaf-system/components/f16llc.py

import numpy as np
from helpers import lqr


def model_output(model, time_t, state_controller, input_all):
    """ get the reference commands for the control surfaces """
    compute_fcn, *trim_points = getattr(lqr, model.lqr_name)()

    assert len(input_all) == 21
    #TODO: hard coded indices!
    x_f16, _y, u_ref = input_all[:13], input_all[13:17], input_all[17:]
    x_ctrl = get_x_ctrl(trim_points, np.concatenate([x_f16, state_controller]))

    # Initialize control vectors
    u_deg = np.zeros((4,))  # throt, ele, ail, rud
    u_deg[1:4] = compute_fcn(x_ctrl)

    # Set throttle as directed from output of getOuterLoopCtrl(...)
    u_deg[0] = u_ref[3]

    # Add in equilibrium control
    u_deg[0:4] += trim_points[1]
    u_deg = clip_u(model, u_deg)

    return u_deg


def model_state_update(model, time_t, state_controller, input_all):
    """ get the derivatives of the integrators in the low-level controller """
    _compute_fcn, *trim_points = getattr(lqr, model.lqr_name)()
    

### Creating a control system

From collection of components, we can build a full control system. The control system is again defined by a simple TOML configuration that describes the interconnections between components. 

In [ ]:
!cat /csaf-system/f16_shield_config.toml

### Loading the configuration

In [6]:
import csaf.config as cconf
import csaf.system as csys

# create a csaf configuration out of toml
my_conf = cconf.SystemConfig.from_toml("/csaf-system/f16_shield_config.toml")

ModuleNotFoundError: No module named 'csaf'

### Display the system topology

In [ ]:
from IPython.display import Image

import pathlib

plot_fname = f"pub-sub-plot.png"

# plot configuration pub/sub diagram as a file -- proj specicies a dot executbale and -Gdpi is a valid dot
# argument to change the image resolution
my_conf.plot_config(fname=pathlib.Path(plot_fname).resolve(), prog=["dot", "-Gdpi=400"])

# display written file to notebook
Image(plot_fname, height=600)

### Simulating the system

In [ ]:
# create pub/sub components out of the configuration
my_system = csys.System.from_config(my_conf)

simulation_timespan = [0, 35.0]

# simulate and collect time traces out of the components
trajs = my_system.simulate_tspan(simulation_timespan, show_status=True)

# destroy components and unbind all used sockets
my_system.unbind() 

### F16 flight animation

In [ ]:
# if you want to use the notebook engine needed to play animations in the notebook
# uncomment the line below:
# %matplotlib notebook
import sys
sys.path.append('/csaf-system')
from f16_plot import plot3d_anim
video = plot3d_anim(trajs["plant"])    

In [ ]:
# if the animation doesn't play well--translate the animation
# object to a JS/HTML video and display it
from IPython.display import HTML
HTML(video.to_jshtml())

### 2D plot of the F16 model

In [ ]:
from f16_plot import plot_simple
plot_simple(trajs)

### Plot the state variables

In [1]:
import matplotlib.pyplot as plt 
import numpy as np

# select component to plot
component_name = "plant"

# select topic of component to plot
topic_name = "states"

if component_name in trajs:
    # time trace of component
    ttrace = trajs[component_name]
    
    if not hasattr(ttrace, topic_name):
        raise RuntimeError(f"ERROR! Invalid topic name {topic_name} for component {component_name}")
    
    # collect time and data to plot
    t, data = ttrace.times, np.array(getattr(ttrace, topic_name))
    
    # number of dimensions -> number of plots
    n_dim = data.shape[1]
    
    # get full component name
    component_vname = my_conf.get_component_settings(component_name)["config"]["system_name"]
    
    # get names of topic from ROSmsg -- skip boilerplate
    names = my_conf.get_msg_setting(component_name, topic_name, "msg").fields_no_header
    
    # create matplotlib axes and plot
    fig, axs = plt.subplots(figsize=(12/2, n_dim*4/2),nrows=n_dim, sharex=True)
    for idx, ax in enumerate(axs):
        # plot formatting
        ax.plot(t, data[:, idx], LineWidth=2)
        ax.set_ylabel(names[idx])
        ax.set_xlim(min(t), max(t))
        ax.grid()
        
    # set figure title
    axs[0].set_title(f"{component_vname} - Topic \"{topic_name.title()}\"")
    
    # on last axis, set the time label
    axs[-1].set_xlabel("time (s)")
else:
    raise RuntimeError(f"ERROR! Invalid component name {component_name}")

NameError: name 'trajs' is not defined

### Replay simulation in FlightGear

If you have [FlightGear installed](file://../../examples/f16/flightgear/FLIGHTGEAR.md), start it on your host with `examples/f16/flightgear/launch_fg.sh`
Once FlightGear fully loads, we can replay the simulation in real-time.

In [ ]:
# uncomment to run
#from f16_plot import render_in_flightgear
#render_in_flightgear(trajs)

## Get Component Signals

This shows how to get the input and signals associated with a component identified by a component name. The function `get_component_io` is implemented, get the input topics of the controller, then calculating what the input buffer of the component is at each time step.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csaf.trace as ctr

# get io of controller
cio = ctr.get_component_io("controller", trajs, my_conf)

# transform f16 state for controller input
xequil = np.array([502.0, 0.03887505597600522, 0.0, 0.0, 0.03887505597600522, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1000.0, 9.05666543872074])
uequil = np.array([0.13946204864060271, -0.7495784725828754, 0.0, 0.0])
x_delta = np.array(np.hstack((trajs["plant"].states.copy(), trajs["controller"].states.copy())))
x_delta[:, :len(xequil)] -= np.tile(xequil, (len(trajs["plant"].states), 1))

# llc f16 inputs/ outputs
outs = np.array(cio["outputs"])[:, 1:4]
ins = x_delta[:, (1, 7, 13, 2, 6, 8, 14, 15)]

# uncomment to save to txt files
#np.savetxt("output_csaf.txt", outs)
#np.savetxt("input_csaf.txt", ins)

In [ ]:
plt.title("Controller Input Signals for Autopilot")
plt.plot(outs[:, :1])
plt.grid()
plt.show()

In [ ]:
# put in dataframe
input_fields = [(p, t) for p, t in my_conf._config["components"]["controller"]["sub"]]
in_fields = np.concatenate([[f"input-{p}-{f}" for f in my_conf.get_msg_setting(p, t, "msg").fields_no_header] for p, t in input_fields])
out_fields = [f"output-controller-{f}" for f in my_conf.get_msg_setting("controller", "outputs", "msg").fields_no_header]
columns = np.concatenate([["times"], in_fields, out_fields])
df = pd.DataFrame(columns=columns, data=np.hstack((cio["times"][:, np.newaxis], cio["inputs"], cio["outputs"])))
#df.to_csv("controller-llc-traces.csv", index=False)